In [ ]:
    import os
    import sys
    import time
    import torch
    import numpy as np

In [ ]:
    #sys.path.insert(1, os.path.join("C:\\", "Soham", "Git", "spectral-NN", "source_codes"))
    sys.path.insert(1, os.path.join("C:\\", "Users", "Soham", "Git", "spectral-NN", "source_codes"))
    #sys.path.insert(1, os.path.join("/home", "soham", "GitHub", "spectral-NN", "source_codes"))

    import SpectralNetworks as spectNN
    import Important_functions as Ifn
    import spectral_NN_setup as setup

In [ ]:
    dirc = setup.directory
#    replicates = setup.replicates
    repl = 1

In [ ]:
        print('Example'+str(repl+1)+':')
        u = np.loadtxt(dirc+"locations.dat",dtype="float32")
        if len(u.shape)==1:
            D, d = len(u), 1
            u = u.reshape(D,1)
        else:
            D, d = u.shape
        u = torch.from_numpy(u)
        x = np.loadtxt(dirc+'Example'+str(repl+1)+'.dat',dtype='float32')
        N = x.shape[0]
        if x.shape[1] != D:
            exit('Data shape mismatch!! Aborting..')
        print('N='+str(N)+', D='+str(D)+', d='+str(d))

        x = torch.from_numpy(x)
        x = x - torch.mean(x,dim=0,keepdim=True)

In [ ]:
        method = "deep"
        M, L = 10, 10
        depth, width = 3, 20
        q = 10
        
        if method.lower()=="shallow":
            model = spectNN.spectralNNShallow(N,d,M,L,setup.act_fn,setup.init)
        elif method.lower()=="deep":
            model = spectNN.spectralNNDeep(N,d,M,L,depth,width,setup.act_fn,setup.init)
        elif method.lower()=="deepshared1":
            model = spectNN.spectralNNDeepshared1(N,d,M,L,depth,width,setup.act_fn,setup.init)
        elif method.lower()=="deepshared2":
            model = spectNN.spectralNNDeepshared2(N,d,M,L,depth,width,setup.act_fn,setup.init)
        elif method.lower()=="deepshared3":
            model = spectNN.spectralNNDeepshared3(N,d,M,L,depth,width,setup.act_fn,setup.init)
        else:
            exit("Undefined model specified! Aborting...")

In [ ]:
        optimizer = setup.optimizer(model.params,lr=setup.lr)
        loss = Ifn.loss_spectralNN(N, setup.wt_fn, grid_size=setup.loss_grid, q=q)
        print("Fitting the model ...")
        start_time = time.time()
        Ifn.spect_NN_optimizer(x,u,model,loss,optimizer,epochs=setup.epochs)
        time_ellapsed = time.time() - start_time
        print("Model fitted. Time taken: {} seconds" .format(time_ellapsed))

In [ ]:
        with torch.no_grad():
            num = loss.loss_fn(x,model(u)).item()
            den = loss.loss_fn(x,0*x).item()
            train_err = num/den
        print("Relative training error: {:.2f}%" .format(train_err*100))
        print("Numerator: {:.4f}, Denominator: {:.4f}" .format(num,den))

        #spect_dens_est = Ifn.spectral_density_evaluation(model, q=q, wt_fn=setup.wt_fn)
        #theta_file = dirc+"True_thetas"+str(repl+1)+".dat"
        #loc_file = dirc+"True_locations"+str(repl+1)+".dat"
        #spect_file = dirc+"True_spectrum"+str(repl+1)+".dat"

        #test_err,num,den,tr_cospect,tr_quadspect,err_cospect,err_quadspect = Ifn.spectral_error_computation(spect_dens_est,theta_file,loc_file,spect_file)

        #print("Relative test error: {:.2f}%" .format(test_err*100))
        #print("Numerator: {:.4f}, Denominator: {:.4f}" .format(num,den))
        #print("Real part: Error - {:.4f}, Actual - {:.4f}" .format(err_cospect,tr_cospect))
        #print("Imaginary part: Error - {:.4f}, Actual - {:.4f}" .format(err_quadspect,tr_quadspect))

In [ ]:
        true_loc = np.loadtxt(dirc+"True_locations_grid.dat",dtype="float32")
        if d == 1:
            true_loc = true_loc.reshape(-1,1)
        D_tr = true_loc.shape[0]
        true_loc = torch.from_numpy(true_loc)
        print(D_tr)

        spect_dens_est = Ifn.spectral_density_evaluation(model, q=q, wt_fn=setup.wt_fn)
        emp_spect_dens = Ifn.empirical_spectral_density(x, q=q, wt_fn=setup.wt_fn)
        true_thetas = np.loadtxt(dirc+"True_thetas_grid.dat",dtype="float32")

        K_theta = len(true_thetas)
        spect_tr = np.loadtxt(dirc+"True_spectrum_grid.dat",dtype="float32")
        spect_tr.shape

        import matplotlib.pyplot as plt

        def mat_scale(mat):
            m = np.max(np.abs(mat))
            if m == 0.:
                return 0.0001
            return m

In [ ]:
for t, theta in enumerate(true_thetas):
    cospect_tr = spect_tr[t*D_tr:(t+1)*D_tr,:D_tr]
    quadspect_tr = spect_tr[t*D_tr:(t+1)*D_tr,D_tr:]
    theta = true_thetas[t]
    cospect_fit, quadspect_fit = spect_dens_est.evaluate_grid(theta,true_loc)
    cospect_emp, quadspect_emp = emp_spect_dens.evaluate(theta)
    
    fig, axes = plt.subplots(figsize=(12,72),ncols=6)
    ax = axes[0]
    m_ = mat_scale(cospect_tr)
    ax.imshow(cospect_tr/m_, origin='lower', cmap='seismic',vmin=-1,vmax=1)
    ax.set_xticks(np.linspace(0,D_tr-1,5))
    ax.set_xticklabels([])
    ax.set_yticks(np.linspace(0,D_tr-1,5))
    ax.set_yticklabels([0,0.25,0.5,0.75,1])
    ax.set_title("True ({:.3f})" .format(m_))
    ax.set_xticklabels([0,0.25,0.5,0.75,1])

    ax = axes[1]
    m_ = mat_scale(cospect_emp.numpy())
    ax.imshow(cospect_emp/m_, origin='lower', cmap='seismic',vmin=-1,vmax=1)
    ax.set_xticks(np.linspace(0,D-1,5))
    ax.set_xticklabels([])
    ax.set_yticks(np.linspace(0,D-1,5))
    ax.set_yticklabels([0,0.25,0.5,0.75,1])
    ax.set_title("Empirical ({:.3f})" .format(m_))
    ax.set_xticklabels([0,0.25,0.5,0.75,1])

    ax = axes[2]
    m_ = mat_scale(cospect_fit.numpy())
    ax.imshow(cospect_fit/m_, origin='lower', cmap='seismic',vmin=-1,vmax=1)
    ax.set_xticks(np.linspace(0,D_tr-1,5))
    ax.set_xticklabels([])
    ax.set_yticks(np.linspace(0,D_tr-1,5))
    ax.set_yticklabels([0,0.25,0.5,0.75,1])
    ax.set_title("Fitted ({:.3f})" .format(m_))
    ax.set_xticklabels([0,0.25,0.5,0.75,1])

    ax = axes[3]
    m_ = mat_scale(quadspect_tr)
    ax.imshow(quadspect_tr/m_, origin='lower', cmap='seismic',vmin=-1,vmax=1)
    ax.set_xticks(np.linspace(0,D_tr-1,5))
    ax.set_xticklabels([])
    ax.set_yticks(np.linspace(0,D_tr-1,5))
    ax.set_yticklabels([0,0.25,0.5,0.75,1])
    ax.set_title("True ({:.3f})" .format(m_))
    ax.set_xticklabels([0,0.25,0.5,0.75,1])

    ax = axes[4]
    m_ = mat_scale(quadspect_emp.numpy())
    ax.imshow(quadspect_emp/m_, origin='lower', cmap='seismic',vmin=-1,vmax=1)
    ax.set_xticks(np.linspace(0,D-1,5))
    ax.set_xticklabels([])
    ax.set_yticks(np.linspace(0,D-1,5))
    ax.set_yticklabels([0,0.25,0.5,0.75,1])
    ax.set_title("Empirical ({:.3f})" .format(m_))
    ax.set_xticklabels([0,0.25,0.5,0.75,1])

    ax = axes[5]
    m_ = mat_scale(quadspect_fit.numpy())
    ax.imshow(quadspect_fit/m_, origin='lower', cmap='seismic',vmin=-1,vmax=1)
    ax.set_xticks(np.linspace(0,D_tr-1,5))
    ax.set_xticklabels([])
    ax.set_yticks(np.linspace(0,D_tr-1,5))
    ax.set_yticklabels([0,0.25,0.5,0.75,1])
    ax.set_title("Fitted ({:.3f})" .format(m_))
    ax.set_xticklabels([0,0.25,0.5,0.75,1])

    plt.show()